In [ ]:
from IPython.core.display import HTML
# Most of this lecture is stolen from Cary, a little from Eric and some other scattered pieces

# What is web scraping?

The practice of gathering data through any means other than a program interacting with an API.

# Why do we web scrape?

Browsers are good for many things, but not so much for gathering information

- Sometimes the data we want isn't available packaged all nicely
- Sometimes the company doesn't want to make it easy for us to get all their data
- Sometimes the data source we need isn't big enough to warrant that company creating an API

# How do we do it?

If you can view it in your browser, you can grab it with a python script!

We'll discuss what packages you need when we get there.

# Is there an easier way?

API's are a thing. Sometimes they give us what we need and that's nice.

# What are the keys to a good scraper?

Some things to keep in mind when building your scraper:

- Save all the data you collect
- Don't abuse try:excepts
- Don't get banned from the site you're interested in

## Afternoon Objectives

1. Understand the process of getting data from the web.
2. Know the basics of HTML/CSS:
    * Know how to pull desired data from web pages.
3. Be able to use existing API's to get fetch pre-formatted data.

### Internet vs. World Wide Web

* The internet is commonly refered to as a network of networks. It is the infrastructure that allows networks all around the world to connect with one another. There are many different protocols to transfer information within this larger, meta-network.
* The World Wide Web, or Web, provides one of the ways that data can be transfered over the internet. Uses a **U**niform **R**esource **L**ocator, URL, to specify the location, within the internet, of a document.
* Documents on the web are generally written in **H**yper**T**ext **M**arkup **L**anguage, HTML, which can be natively viewed by browsers, the tool that we use to browse the web.

### Communication on the Web

Information is transmitted around the web through a number of protocols. The main one that you will see is the **H**yper**T**ext **T**ransfer **P**rotocol, HTTP. These transfers, called **requests**, are initiated in a number of ways, but always begin with the client, read: you at your browser.

There are 4 main types of request that can be issued by your browser: get, post, put and delete. For web scraping purposes, you will almost always be using get requests. We will learn some more about the others in a couple of weeks during data products day.

### HTML & CSS
HTML, or *HyperText Markup Language*, is the language that virtually all web pages are written in.  HTML allows us to describe how web pages should render themselves and display content.  They do this by using *HTML Tags*, where each tag is describing different document content.  For example we could specify the title of our page with the following tags: `<title>This is the Title!</title>`.  `<title>` is the opening tag and `</title>` is the corresponding closing tag.  Anything that falls in the middle will be interpreted as the title of the document.  For a more in-depth introduction to HTML, please refer to [this w3schools Tutorial](http://www.w3schools.com/html/html_intro.asp).

We can also apply *attributes* to HTML tags to control how they will be stylized.  Let's say we would like to change the color of our super original title tag to red to make it pop.  We could easily achieve this with the following style attribute: `<title style="color:red;">This is a red Title!</title>`.  But what if we have a very large website and want to change the look and feel of the entire thing?  Certainly there has to be a better way than manually manipulating individual tags?  

This can easily be achieved with the use of *Cascading Style Sheets*, or CSS.  Typically CSS is defined in a separate style sheet file with a `.css` extension which controls how certain tags are stylized.  For example, the following CSS block would make any `<p>` tags have red text and be in the courier font:

```css
p  {
    color: red;
    font-family: courier;
}
```

This is all well and good, but our goal isn't to build websites, it's to systematically extract information from them!  To this end, CSS selectors provide a succinct way of only selecting certain aspects from a potentially giant block of HTML.

#### CSS Selectors
To really see the power of CSS selectors we need to see them in action!  To that end, try getting through the first 10-15 levels of the [Game of Fruit](http://flukeout.github.io/).

You can also refer to this [CSS Selector Cheatsheet](http://www.cheetyr.com/css-selectors)

# Scraping from a Web Page with Python

Scraping a web site basically comes down to making a request from Python and parsing through the HTML that is returned from each page. For each of these tasks we have a Python library, `requests` and `bs4`, respectively.

### Requests Library

The [requests](http://docs.python-requests.org/en/latest/index.html) library is designed to simplify the process of making http requests within Python. The interface is mindbogglingly simple. Instantiate a requests object to the request, this will mostly be a `get`, with the URL and optional parameters you'd like passed through the request. That instance make the results of the request available via attributes/methods.

In [1]:
import requests
fun_cheap = 'http://sf.funcheap.com'
r = requests.get('http://sf.funcheap.com/2016/06/25/')

In [2]:
r.text[:1000] # First 1000 characters of the HTML

u'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\r\n<html xmlns="http://www.w3.org/1999/xhtml" lang="en-US" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraphprotocol.org/schema/">\r\n\r\n<head profile="http://gmpg.org/xfn/11">\r\n<script src="//cdn.optimizely.com/js/195632799.js"></script>\r\n\r\n<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />\r\n\r\n\r\n<title>Events for June 25, 2016 Archives - FunCheapSF.com</title>\r\n\r\n<meta name="generator" content="WordPress" /> <!-- leave this for stats -->\r\n\r\n<link rel="stylesheet" href="http://cdn.funcheap.com/wp-content/themes/arthemia-premium/style.css?v=1.6" type="text/css" media="screen" />\r\n<link rel="stylesheet" href="http://cdn.funcheap.com/wp-content/themes/arthemia-premium/madmenu.css?v=1.1" type="text/css" media="screen" />\r\n<!--[if IE 6]>\r\n    <style type="text/css">\r\n    body {\r\n        behavior:ur

### Getting Info from a Web Page with BeautifulSoup

Now that we can gain easy access to the HMTL for a web page, we need some way to pull the desired content from it. Luckily there is already a system in place to do this. With a combination of HMTL and CSS selectors we can identify the information on a HMTL page that we wish to retrieve and grab it with [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#searching-the-tree).

“Beautiful Soup, so rich and green,

Waiting in a hot tureen!

Who for such dainties would not stoop?

Soup of the evening, beautiful Soup!”

--Lewis Carroll

In [3]:
import requests
import re
from bs4 import BeautifulSoup

In [4]:
r = requests.get('http://www.espn.com/college-football/statistics/player/_/stat/passing/sort/passingYards/year/2015/qualified/false/count/1')

In [5]:
bs_obj = BeautifulSoup(r.text, 'html.parser')

In [6]:
bs_obj.findAll('tr',{'class':'evenrow','class':'oddrow'})
#DOESNT WORK EVEN THOUGH TEXTBOOK SAYS IT SHOULD!

[<tr align="right" class="oddrow player-23-504866"><td align="left">1</td><td align="left"><a href="http://www.espn.com/college-football/player/_/id/504866/brandon-doughty">Brandon Doughty</a>, QB</td><td align="left"><span title="Western Kentucky">WKU</span></td><td>388</td><td>540</td><td>71.9</td><td class="sortcell">5055</td><td>9.4</td><td>78</td><td>48</td><td>9</td><td>15</td><td>176.5</td></tr>,
 <tr align="right" class="oddrow player-23-547401"><td align="left">3</td><td align="left"><a href="http://www.espn.com/college-football/player/_/id/547401/jared-goff">Jared Goff</a>, QB</td><td align="left"><span title="California">CAL</span></td><td>341</td><td>529</td><td>64.5</td><td class="sortcell">4719</td><td>8.9</td><td>80</td><td>43</td><td>13</td><td>26</td><td>161.3</td></tr>,
 <tr align="right" class="oddrow player-23-550629"><td align="left">5</td><td align="left"><a href="http://www.espn.com/college-football/player/_/id/550629/luke-falk">Luke Falk</a>, QB</td><td align="l

In [10]:
bs_obj.findAll('tr',{'class':re.compile('^(evenrow|oddrow)')})

[<tr align="right" class="oddrow player-23-504866"><td align="left">1</td><td align="left"><a href="http://www.espn.com/college-football/player/_/id/504866/brandon-doughty">Brandon Doughty</a>, QB</td><td align="left"><span title="Western Kentucky">WKU</span></td><td>388</td><td>540</td><td>71.9</td><td class="sortcell">5055</td><td>9.4</td><td>78</td><td>48</td><td>9</td><td>15</td><td>176.5</td></tr>,
 <tr align="right" class="evenrow player-23-513573"><td align="left">2</td><td align="left"><a href="http://www.espn.com/college-football/player/_/id/513573/matt-johnson">Matt Quinn Johnson</a>, QB</td><td align="left"><span title="Bowling Green">BGSU</span></td><td>383</td><td>569</td><td>67.3</td><td class="sortcell">4946</td><td>8.7</td><td>94</td><td>46</td><td>8</td><td>36</td><td>164.2</td></tr>,
 <tr align="right" class="oddrow player-23-547401"><td align="left">3</td><td align="left"><a href="http://www.espn.com/college-football/player/_/id/547401/jared-goff">Jared Goff</a>, QB<

In [15]:
for obj in bs_obj.findAll('tr',{'class':re.compile('^(evenrow|oddrow)')}):
    print obj.find('span')['title']

Western Kentucky
Bowling Green
California
Texas Tech
Washington State
Southern Mississippi
Georgia State
Tulsa
Clemson
Ole Miss
Louisiana Tech
Middle Tennessee
Arizona State
Central Michigan
Mississippi State
Memphis
Oklahoma State
Oklahoma
UCLA
TCU
Indiana
USC
Western Michigan
Arkansas
BYU
Boise State
Miami
West Virginia
Michigan State
Alabama
North Carolina
Nebraska
Michigan
Idaho
Temple
Buffalo
Toledo
Washington
UMass
Notre Dame


In [16]:
for obj in bs_obj.findAll('tr',{'class':re.compile('^(evenrow|oddrow)')}):
    html = requests.get(obj.find('a')['href'])
    bs_obj2 = BeautifulSoup(html.text, 'html.parser')
    name = bs_obj2.h1.get_text()
    born = bs_obj2.find('ul',{'class':'player-metadata'}).findAll('li')[0].get_text()
    if born[0] == 'B':
        print '{} Born: {}'.format(name,born[4:])

Brandon Doughty Born: Oct 6, 1991 in Davie, FL (Age: 25)
Matt Johnson Born: Sep 9, 1992 in Harrisburg, PA
Jared Goff Born: Oct 14, 1994 in San Rafael, CA (Age: 22)
Patrick Mahomes II Born: Sep 17, 1995 in Tyler, TX (Age: 21)
Nick Mullens Born: Mar 21, 1995 in Hoover, AL (Age: 22)
Dane Evans Born: Nov 19, 1993 in Chickasha, OK (Age: 23)
Deshaun Watson Born: Sep 14, 1995 in Gainesville, GA (Age: 21)
Chad Kelly Born: Mar 26, 1994 in Niagara Falls, NY (Age: 23)
Jeff Driskel Born: Apr 23, 1993 in Oviedo, FL (Age: 24)
Mike Bercovici Born: Feb 9, 1993 in Northridge, CA (Age: 24)
Cooper Rush Born: Nov 21, 1993 in Charlotte, MI (Age: 23)
Dak Prescott Born: Jul 29, 1993 in Sulphur, LA (Age: 24)
Paxton Lynch Born: Feb 12, 1994 in San Antonio, TX (Age: 23)
Trevone Boykin Born: Aug 22, 1993 in Mesquite, TX (Age: 23)
Nate Sudfeld Born: Oct 7, 1993 in Modesto, CA (Age: 23)
Cody Kessler Born: May 11, 1993 in Bakersfield, CA (Age: 24)
Zach Terrell Born: May 1, 1993 in Fort Wayne, IN
Brandon Allen Born:

This is an example of a recursive crawler or spider

# Basic Authentication

In [ ]:
import requests
z = requests.get('http://galvanizesf.roomzilla.net')
z

In [ ]:
import requests
z = requests.get('http://galvanizesf.roomzilla.net', auth=('', 'gVIP543'))
z

In [ ]:
HTML(z.content)

Sometimes they hide the info from us, but we can find it being sneaky...
http://m.mlb.com/scoreboard#date=11/1/2016

As you go through a web site you should build up a dictionary for the documents that you want to store in Mongo. In the example above we may, for each post url, create a dictionary with the information:
```python
    { url: url_of_event,
      date: date_event,
      cost: cost_of_event }
```

We can then insert these dictionaries into a Mongo database via PyMongo, which we will learn about next.

# Scraping from an Existing API

Let's take a look at the API for all the publically avaliable policing data in the [UK](https://data.police.uk/docs/). After taking a look at the documentation for the interface, let's experiment with what we get when we issue a request to this API. The process looks remarkable similar to the one we went through for scraping a web page, except this time the response we're looking for is avaliable via the `json()` method.

In [19]:
r = requests.get('https://data.police.uk/api/crimes-street/all-crime?lat=52.629729&lng=-1.131592&date=2013-01')
print r

<Response [404]>


## API Scraping and Mongo

Many APIs will give you a choice of how it will return data to you, choosing json will make life easier since we will frequently be using Mongo for our storage unit during our scraping endeavors, and it plays very well with json. 

Interacting with Mongo from Python is done with the other Mongo client that we talked about earlier PyMongo. It is designed to have a similar interface as the Mongo shell does, this ends up being fairly intuitive since both Python and JavaScript are object oriented languages, and therefore store and refer to things in a similar manner.

In [20]:
from pymongo import MongoClient

client = MongoClient()
db = client.uk_police
collection = db.all_crime

In [21]:
other_request = requests.get('https://data.police.uk/api/crimes-no-location?category=all-crime&force=warwickshire&date=2013-09')

In [22]:
other_request.json()[:2]

ValueError: No JSON object could be decoded

In [ ]:
# Possible way to grab data for range of months and years
for year in range(2001, 2016):
    for month in range(1, 13):
        r = requests.get('https://data.police.uk/api/crimes-no-location?category=all-crime&force=warwickshire&date={}-{}'.format(year, month))
        collection.insert_many(r.json())

In [ ]:
collection.insert_many(other_request.json())

In [ ]:
import pprint as pp
for item in collection.find({ 'category' : 'public-order' }):
    pp.pprint(item)

In [ ]:
# Remember to close the connection
client.close()

# Wikipedia API

In [23]:
# import the Requests HTTP library
import requests
import json
import re

# A User agent header required for the Wikipedia API.
headers = {'user_agent': 'Web_Scraping/1.1 (darren.reger@galvanize.com; dsi example exercise)'}

In [24]:
# Experiment with fetching one or two pages and examining the result (fill in URL and payload)
url = 'https://en.wikipedia.org/w/api.php'

# parameters for the API request
payload = { 'action' : 'parse' , 'format' : 'json','page' : 'Kevin Bacon' }

# make the request
r = requests.post(url, data=payload, headers=headers)

# print out the result of the request as JSON
print r.json()['parse']

# this is the same as in Chrome going to https://en.wikipedia.org/w/api.php?action=parse&format=json&page=Kevin%20Bacon

{u'templates': [{u'*': u'Template:Other uses', u'ns': 10, u'exists': u''}, {u'*': u'Template:Pp-semi-blp', u'ns': 10, u'exists': u''}, {u'*': u'Template:Pp-blp', u'ns': 10, u'exists': u''}, {u'*': u'Template:Use mdy dates', u'ns': 10, u'exists': u''}, {u'*': u'Template:DMCA', u'ns': 10, u'exists': u''}, {u'*': u'Template:Dated maintenance category', u'ns': 10, u'exists': u''}, {u'*': u'Template:FULLROOTPAGENAME', u'ns': 10, u'exists': u''}, {u'*': u'Template:Ns has subpages', u'ns': 10, u'exists': u''}, {u'*': u'Template:Infobox person', u'ns': 10, u'exists': u''}, {u'*': u'Template:Infobox', u'ns': 10, u'exists': u''}, {u'*': u'Template:Br separated entries', u'ns': 10, u'exists': u''}, {u'*': u'Template:PAGENAMEBASE', u'ns': 10, u'exists': u''}, {u'*': u'Template:Birth date and age', u'ns': 10, u'exists': u''}, {u'*': u'Template:MONTHNAME', u'ns': 10, u'exists': u''}, {u'*': u'Template:MONTHNUMBER', u'ns': 10, u'exists': u''}, {u'*': u'Template:Age', u'ns': 10, u'exists': u''}, {u'*'

In [25]:
print r.text

{"parse":{"title":"Kevin Bacon","pageid":16827,"revid":793735268,"text":{"*":"<div class=\"mw-parser-output\"><div role=\"note\" class=\"hatnote navigation-not-searchable\">For other uses, see <a href=\"/wiki/Kevin_Bacon_(disambiguation)\" class=\"mw-disambig\" title=\"Kevin Bacon (disambiguation)\">Kevin Bacon (disambiguation)</a>.</div>\n<table class=\"infobox biography vcard\" style=\"width:22em\">\n<tr>\n<th colspan=\"2\" style=\"text-align:center;font-size:125%;font-weight:bold\"><span class=\"fn\">Kevin Bacon</span></th>\n</tr>\n<tr>\n<td colspan=\"2\" style=\"text-align:center\"><a href=\"/wiki/File:Kevin_Bacon_SDCC_2014.jpg\" class=\"image\"><img alt=\"Kevin Bacon SDCC 2014.jpg\" src=\"//upload.wikimedia.org/wikipedia/commons/thumb/d/d7/Kevin_Bacon_SDCC_2014.jpg/220px-Kevin_Bacon_SDCC_2014.jpg\" width=\"220\" height=\"306\" srcset=\"//upload.wikimedia.org/wikipedia/commons/thumb/d/d7/Kevin_Bacon_SDCC_2014.jpg/330px-Kevin_Bacon_SDCC_2014.jpg 1.5x, //upload.wikimedia.org/wikipedi

# Downloading an image or file
We use urlretrieve

In [ ]:
from urllib import urlretrieve

In [ ]:
urlretrieve('https://images.craigslist.org/00Y0Y_kho6IzfhxVn_600x450.jpg', filename='cltest.jpg')

## Leftover from Cary's Lecture in case the ESPN live demo goes poorly

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(r.text, 'html.parser')

In [ ]:
soup.find_all('a', rel=True)[:10]

In [ ]:
soup.select('h2.title')

In [ ]:
title = soup.find_all('h2', class_='title')[0]

In [ ]:
good_clear_float = title.next_sibling.next_sibling

In [ ]:
urls = []
for tag in good_clear_float.find_all('a', rel=True):
    urls.append(tag.attrs['href'])

In [ ]:
urls